# Video Processing for BT Lab
Corrects png files
<br>
<br>
Notes:
1) looking for a specific path 
2) need to enter drive name
<br>
<br>
![UofC logo](./pictures/uofc_logo-black.jpg)

In [1]:
# import libraries
import os
import re
import datetime
import platform
import time

In [2]:
# declare basic filepaths
repo_dir = os.getcwd()
log_dir = repo_dir + "/records/logs"

In [3]:
# given a list of filepaths, create a list of unique directories

def find_png_dir_from_log_file(log_file:str):
    png_directories = []
    
    with open(log_file, "r") as log_data:
        log_files = log_data.read().split("\n")
    
    for log_file in log_files:
        log_file_temp = log_file.replace("\\", "/")
        log_file_split = log_file_temp.split("/")
        
        if "" in log_file_split:
            log_file_split.remove("")

        video_folder = "/".join(log_file_split[:-1])

        if video_folder not in png_directories:
            png_directories.append(video_folder)

    return png_directories

In [4]:
# write to txt file

def save_folder(list_of_paths:list, log_filename:str) -> None:
    if os.path.exists(log_filename):
        os.remove(log_filename)
    
    with open(log_filename, "a") as log_data:
        for folder in list_of_paths:
            log_data.write(f"{folder}\n")

In [5]:
""" gets the delta time of images based on creation time """

def get_file_creation_time(filepath):
    ctime = os.path.getctime(filepath)
    return datetime.datetime.fromtimestamp(ctime)

def get_min_max_times(filepaths):
    creation_times = [get_file_creation_time(fp) for fp in filepaths]
    return min(creation_times), max(creation_times)

def get_delta_time(png_files: list[str]):
    min_time, max_time = get_min_max_times(png_files)
    delta_time = (max_time - min_time).total_seconds()
    return delta_time


In [6]:
def extract_info_from_file_path(filepaths: list[str]):
    all_videos = []
    file_info_from_path = {
        0 : "ID",
        1 : "distance",
        2 : "blanket",
        3 : "breathing",
        4 : "filename"
    }

    video_info = {
        "frames" : None,
        "length" : None,
        "old fps" : None,
        "alias": None,
        "local path" : None,
    }

    splice_stop = 5

    for filepath in filepaths:
        try:
            filepath_to_list = filepath.split("/")

            # extract info from filepath
            filepath_to_list = filepath_to_list[-splice_stop:]
            patient_info = {tag:filepath_to_list[key] for key, tag in file_info_from_path.items() if key<splice_stop}
            
            # combine information from filepath and default parameters
            patient_full_info = {**patient_info, **video_info}
            patient_full_info["local path"] = filepath

            all_videos.append(patient_full_info) # save to list
        except Exception as e:
            print(f"{type(e)} from {filepath}")

        break

    return all_videos

## Retrieve all videos (stage 3.2)

Correct png files

In [7]:
"get png folders (rgb)"
log_filename = log_dir +"/DI_CAMERA_P3225-png.log"
logfile_rgb_save = log_dir + "/all_png/rgb_folders-png.log"
rgb_png_folders = find_png_dir_from_log_file(log_filename)
save_folder(rgb_png_folders, logfile_rgb_save)

In [8]:
"get png folders (thermal)"
log_filename = log_dir +"/DI_THERMAL_FLIR-png.log"
logfile_thermal_save = log_dir + "/all_png/thermal_folders-png.log"
thermal_png_folders = find_png_dir_from_log_file(log_filename)
save_folder(thermal_png_folders, logfile_thermal_save)

In [9]:
thermal_png_folders

['E:/Christian/DI_centre_structured/DI_THERMAL_FLIR/15/15_1/2 meters/WB/H/frames',
 'E:/Christian/DI_centre_structured/DI_THERMAL_FLIR/15/15_2/2 Meters/WB/H/frames',
 'E:/Christian/DI_centre_structured/DI_THERMAL_FLIR/15/15_2/2 Meters/WB/R/frames',
 '']

In [10]:
""" extract info from file path """

metadata_thermal = extract_info_from_file_path(thermal_png_folders)
metadata_thermal[0]

{'ID': '15_1',
 'distance': '2 meters',
 'blanket': 'WB',
 'breathing': 'H',
 'filename': 'frames',
 'frames': None,
 'length': None,
 'old fps': None,
 'alias': None,
 'local path': 'E:/Christian/DI_centre_structured/DI_THERMAL_FLIR/15/15_1/2 meters/WB/H/frames'}

In [11]:
""" extract info from file path """

# test_path = ""
# os.chdir(test_path)
# png_files = [png_file for png_file in os.listdir(test_path) if png_file.endswith(".png")]
# num_of_files = len(png_files)
# delta_time = get_delta_time(png_files)
# old_fps = num_of_files/delta_time

' extract info from file path '